# Countdown Numbers Game

***

## Example & Explanation
Example and Explanation taking from https://wiki.apterous.org/Numbers_game
***
The Countdown numbers Game is a popular televised gameshow game, this notebook is a replication of the technical and logical side of countdown numbers game. The countdown numbers game involves a board of 24 numbered cards, arranged faced down. The top row contains the large numbers which are 25, 50, 75, 100. The three other rows contain cards with small numbers between 1 and 10 inclusive. A contestant most chose six cards from the 24 cards available to them, the contestant has the option to choose the amount of "large" or "small" numbered cards, for example they could chose 1 large numbered card and 5 small numbered cards (75, 3, 2, 1, 7, 4). Once the 6 cards are chosen by the contestant, a random 3-digit number is generated called the target number, the target can range from 101 to 999. The contestant has 30 seconds to use the 6 numbers they chose to make the target number or as near as possible.    

#### Game Example
Six Chosen Numbers: 
$
\begin{align}
  75, 2, 5, 6, 1, 4
\end{align}
$ 

Random Three Digit Target Number: 
$
\begin{align}
 273
\end{align}
$ 

Solution:
$
\begin{align}
 75 × 4 = 300
\end{align}
$ 

$
\begin{align}
 5 × 6 − 2 − 1 = 27
\end{align}
$ 

$
\begin{align}
 300 − 27 = 273
\end{align}
$

The Countdown Numbers Game in this notebook follows the rules that exist in the original countdown numbers game. However, one small change is that the six numbers the player originally was able to choose himself are randomly chosen by the computer. Other than that, the game functions just like the original and follows all rules and concepts correctly.

## Overview of Countdown Numbers Game

***

In [1]:
#Imports

# Permutations and combinations.
import itertools as it

# Random number generation.
import random

# Operators as functions.
import operator

In [2]:
# Generate Random Game Numbers
def new_numbers_game(no_large=None):
  # Returns six numbers and a target number representing a Countdown numbers game.
  
  # If no_large in None, randomly pick value between 0 and 4 inclusive.
  if no_large is None:
    # Randomly set the value.
    no_large = random.randrange(0, 5)
  
  # Select random large numbers.
  large_rand = random.sample([25, 50, 75, 100], no_large)
  # Select random small numbers.
  small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
  # The playing numbers.
  play_nos = large_rand + small_rand

  # Select a target number.
  target = random.randrange(101, 1000)

  # Return the game.
  return play_nos, target

In [3]:
# Random nubmers game.
gameNumbers = new_numbers_game()

In [4]:
gameNumbers[0]

[100, 50, 1, 4, 6, 7]

In [5]:
gameNumbers[1]

667

In [6]:
operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]

In [7]:
# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# All pair, op combs that hit target.
for nos, op in it.product(it.permutations(gameNumbers[0], 6), ops):
    #print(nos[0],nos[1],op(nos[0], nos[1]))
    
    if op(nos[0], nos[1]) == 300:
        print(nos[0], str(op), nos[1])

50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
50 <built-in function mul> 6
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in function mul> 50
6 <built-in fu

In [8]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield [*left, *right, operators[0]]

In [9]:
# Evaluate RPN expression.
def eval_rpn(rpn):
  # A stack.
  stack = []
  # Loop through rpn an item at a time.
  for i in rpn:
    # Check if it's a number.
    if isinstance(i, int):
      # Append to the stack.
      stack = stack + [i]
    else:
      # Pop from stack twice.
      right = stack[-1]
      stack = stack[:-1]
      left = stack[-1]
      stack = stack[:-1]
      # Push operator applied to stack elements.
      #print("This is i: ", i)
      if i == operator.truediv and right == 0:
          stack = stack + [0]
      else:
          stack = stack + [i(left, right)]
  # Should only be one item on stack.
  return stack[0]

In [10]:
numbers = [100, 75, 10, 4, 2, 1]

In [11]:
# Using eval, which mightn't be great.
for i in patterns(gameNumbers[0], operators):
  print(eval_rpn(i), i)

-700 [100, 50, 1, 4, 6, 7, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
-700 [100, 50, 1, 4, 6, <built-in function add>, 7, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
-300 [100, 50, 1, 4, <built-in function add>, 6, 7, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
300 [100, 50, 1, 4, 6, <built-in function add>, <built-in function add>, 7, <built-in function sub>, <built-in function mul>, <built-in function add>]
300 [100, 50, 1, 4, <built-in function add>, 6, <built-in function add>, 7, <built-in function sub>, <built-in function mul>, <built-in function add>]
933 [100, 50, 1, <built-in function sub>, 4, 6, 7, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>]
933 [100, 50, 1, <built-in function sub>, 4, 6, <built-in function add>, 7, <built-in funct

In [35]:
def displayCalculation(cal, temp):
    calculation = []
    b = 0
    print("Answer: ", temp, " = ", end="")
    for i in cal:
        if i == operator.truediv:
            calculation = calculation + ["/"]
        elif i == operator.sub:
            calculation = calculation + ["-"]
        elif i == operator.add:
            calculation = calculation + ["+"]
        elif i == operator.mul:
            calculation = calculation + ["*"]
        else:
            calculation = calculation + [i]
            
        print(calculation[b], " ", end="")
        b = b + 1       

In [38]:
# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# Limit the output.
limit = 1100
print("Game Numbers: ", gameNumbers[0])
print("Target: ", gameNumbers[1])
# For the limit.
i = 0
# Orderings of pairs.
for play_nos, opers in it.product(it.permutations(gameNumbers[0]), it.product(*([ops] * 5))):
  #print(play_nos, opers)
  for c in patterns(gameNumbers[0], opers):
      
      evaluatedNumber = eval_rpn(c)
      #print(temp)
      if gameNumbers[1] == evaluatedNumber: 
          #print("Complete", c)
          displayCalculation(c, evaluatedNumber)
          break
  i = i + 1
  if i >= limit:
    break

Game Numbers:  [100, 50, 1, 4, 6, 7]
Target:  667
Answer:  667.0  = 100  50  1  4  +  /  +  6  *  7  +  

## Complexity of Countdown Numbers Game

***

## Functional Programming

https://docs.python.org/3/howto/functional.html
***

### What is Functional Programming?
Functional programming is based on producing simplistic reduced functions that focus on an argument and return value. A function in functional programming is not meant to have side effect, you give the function an input and it will return an output without modifying the inputs. This is known as a pure function where the results of the return value are dependent on the input parameter. Functional programming allows for the creation of clean and maintainable software, keeping the data side and functionality independent, avoiding changing state of variables (it’s okay to declare a new variable instead of re-using existing variables) and treating functions as first class meaning treat functions like data, where a function can be assigned to a variable.

## Functional Aspect of Code behind Countdown Numbers Game.


## References

***

 * Numbers Game https://wiki.apterous.org/Numbers_game
 * Python Functional Programming https://docs.python.org/3/howto/functional.html
 * Matthew Tyson "What is functional programming? Apractical guide" https://www.infoworld.com/article/3613715/what-is-functional-programming-a-practical-guide.html